### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
tot_schools = df["school_name"].nunique()
tot_stu = df["student_name"].count()
hs_df = df.groupby(["school_name"])
tot_budget = hs_df["budget"].max().sum()
av_math = df["math_score"].sum() / df["math_score"].count()
av_read = df["reading_score"].sum() / df["reading_score"].count()
av_score = 0.5 * (av_math + av_read)
df_math_pass = df.loc[df["math_score"] >= 70] #/ tot_stu
pc_math_pass = 100 * (df_math_pass["math_score"].count() / tot_stu)
df_read_pass = df.loc[df["reading_score"] >= 70] #/ tot_stu
pc_read_pass = 100 * (df_read_pass["reading_score"].count() / tot_stu)
data = [[tot_schools,tot_stu,tot_budget,av_math,av_read,av_score,pc_math_pass,pc_read_pass]]
new_df = pd.DataFrame(data, columns = [
    "Total Schools",
    "Total Students",
    "Total Budget",
    "Average Math Score",
    "Average Reading Score",
    "Average Overall Score",
    "Percent Math Pass Rate",
    "Percent Reading Pass Rate"
])
new_df["Total Budget"] = new_df["Total Budget"].map("${:,}".format)
new_df["Average Math Score"] = new_df["Average Math Score"].map("{:.2f}".format)
new_df["Average Reading Score"] = new_df["Average Reading Score"].map("{:.2f}".format)
new_df["Average Overall Score"] = new_df["Average Overall Score"].map("{:.2f}".format)
new_df["Percent Math Pass Rate"] = new_df["Percent Math Pass Rate"].map("{:.2f}%".format)
new_df["Percent Reading Pass Rate"] = new_df["Percent Reading Pass Rate"].map("{:.2f}%".format)

new_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Average Overall Score,Percent Math Pass Rate,Percent Reading Pass Rate
0,15,39170,"$24,649,428",78.99,81.88,80.43,74.98%,85.81%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [3]:
df_schools = df.groupby('school_name').agg({
    'type':'first',
    'student_name': 'count', 
    'budget':'first',
    'math_score':'sum',
    'reading_score':'sum'
}).reset_index()

budget_per = df_schools['budget']/df_schools['student_name']
av_math = df_schools['math_score']/df_schools['student_name']
av_read = df_schools['reading_score']/df_schools['student_name']

tot_stu_per_school = df_schools['student_name']

math_grouped = df.loc[df['math_score'] >= 70].groupby('school_name').count().reset_index()
math_pass = math_grouped['student_name']

read_grouped = df.loc[df['reading_score'] >= 70].groupby('school_name').count().reset_index()
read_pass = read_grouped['student_name']

both_pass = (math_pass + read_pass) / 2

pc_math_pass = math_pass/tot_stu_per_school * 100
pc_read_pass = read_pass/tot_stu_per_school * 100
pc_both_pass = both_pass/tot_stu_per_school * 100

df_schools['budget'] = df_schools['budget'].map("${:,}".format)
df_schools['Budget per Student'] = budget_per.map("${:.0f}".format)
df_schools['Average Math Score'] = av_math.map("{:.2f}".format)
df_schools['Average Reading Score'] = av_read.map("{:.2f}".format)
df_schools['% Passing Math'] = pc_math_pass #.map("{:.2f}%".format)
df_schools['% Passing Reading'] = pc_read_pass #.map("{:.2f}%".format)
df_schools['% Overall Passing Rate'] = pc_both_pass #.map("{:.2f}%".format)
df_schools.drop(['student_name','math_score','reading_score'], axis = 1, inplace = True)
df_schools.rename(columns={"school_name": "School Name",
                           "type": "School Type",
                           "budget": "Total School Budget"},
                            inplace = True)

df_topschools = df_schools.sort_values('% Overall Passing Rate', ascending = False ).reset_index()
df_topschools.head()

,index,School Name,School Type,Total School Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,1,Cabrera High School,Charter,"$1,081,356",$582,83.06,83.98,94.133477,97.039828,95.586652
1,12,Thomas High School,Charter,"$1,043,130",$638,83.42,83.85,93.272171,97.308869,95.290520
2,9,Pena High School,Charter,"$585,858",$609,83.84,84.04,94.594595,95.945946,95.270270
3,4,Griffin High School,Charter,"$917,500",$625,83.35,83.82,93.392371,97.138965,95.265668
4,13,Wilson High School,Charter,"$1,319,574",$578,83.27,83.99,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [4]:
df_bottschools = df_schools.sort_values( '% Overall Passing Rate', ascending = True ).reset_index()
df_bottschools.head()


,index,School Name,School Type,Total School Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,10,Rodriguez High School,District,"$2,547,363",$637,76.84,80.74,66.366592,80.220055,73.293323
1,2,Figueroa High School,District,"$1,884,411",$639,76.71,81.16,65.988471,80.739234,73.363852
2,7,Huang High School,District,"$1,910,635",$655,76.63,81.18,65.683922,81.316421,73.500171
3,8,Johnson High School,District,"$3,094,650",$650,77.07,80.97,66.057551,81.222432,73.639992
4,3,Ford High School,District,"$1,763,916",$644,77.10,80.75,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [5]:
df_grades = df.groupby(['school_name','grade'])

sum = df_grades.sum()
counter = df_grades.count()
sum["Number of Students"] = counter['student_name']

grade_math = sum['math_score'] / sum['Number of Students']
grade_math = grade_math.unstack(1)
grade_math = grade_math[['9th','10th','11th', '12th']]
grade_math.index.names = ['School Name']
grade_math

grade,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [6]:
grade_read = sum['reading_score'] / sum['Number of Students']
grade_read = grade_read.unstack(1)
grade_read = grade_read[['9th','10th','11th', '12th']]
grade_read.index.names = ['School Name']
grade_read

grade,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [7]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [8]:
df_schools.head()
df_schools['Budget per Student'] = df_schools['Budget per Student'].str.replace('$', '').astype(int)

df_schools["Budget Brackets"] = pd.cut(df_schools["Budget per Student"], spending_bins, labels=group_names)
cols = ['Average Math Score','Average Reading Score','% Passing Math','% Passing Reading']
df_schools[cols] = df_schools[cols].apply(pd.to_numeric, errors='coerce')

math_pass = math_grouped[['school_name','student_name']]
math_pass['student_name'] = math_pass['student_name']/tot_stu_per_school * 100
math_pass = math_pass.rename(columns = {'school_name' : 'School Name'})

read_pass = read_grouped[['school_name','student_name']]
read_pass['student_name'] = read_pass['student_name']/tot_stu_per_school * 100
read_pass = read_pass.rename(columns = {'school_name' : 'School Name'})

df_schools_new = pd.merge(df_schools,math_pass,on= 'School Name')
df_schools_new = pd.merge(df_schools_new,read_pass, on='School Name')

both_pass = (math_pass['student_name'] + read_pass['student_name']) / 2
df_schools_new['Overall Passing Rate'] = both_pass

df_schools_new.head()

budg_brack = df_schools_new.groupby('Budget Brackets').agg({
    'Average Math Score':'mean',
    'Average Reading Score':'mean',
    'student_name_x':'mean',
    'student_name_y':'mean',
    'Overall Passing Rate':'mean'
}).reset_index()

budg_brack = budg_brack.rename(columns = {
    'student_name_x':'% Passing Math',
    'student_name_y':'% Passing Reading'
})

budg_brack['Average Math Score'] = budg_brack['Average Math Score'].map("{:.2f}".format)
budg_brack['Average Reading Score'] = budg_brack['Average Reading Score'].map("{:.2f}".format)
budg_brack['% Passing Math'] = budg_brack['% Passing Math'].map("{:.2f}%".format)
budg_brack['% Passing Reading'] = budg_brack['% Passing Reading'].map("{:.2f}%".format)
budg_brack['Overall Passing Rate'] = budg_brack['Overall Passing Rate'].map("{:.2f}%".format)

budg_brack

C:\Users\Natha\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\Natha\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,Budget Brackets,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,<$585,83.45,83.93,93.46%,96.61%,95.04%
1,$585-615,83.60,83.89,94.23%,95.90%,95.07%
2,$615-645,79.08,81.89,75.67%,86.11%,80.89%
3,$645-675,77.00,81.03,66.16%,81.13%,73.65%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
school_sizes = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [10]:
df_schools_new['Tot_Students'] = tot_stu_per_school

df_schools_new["School Size"] = pd.cut(df_schools_new['Tot_Students'], size_bins, labels = school_sizes)

size_brack = df_schools_new.groupby('School Size').agg({
    'Average Math Score':'mean',
    'Average Reading Score':'mean',
    'student_name_x':'mean',
    'student_name_y':'mean',
    'Overall Passing Rate':'mean'
}).reset_index()

size_brack = size_brack.rename(columns = {
    'student_name_x':'% Passing Math',
    'student_name_y':'% Passing Reading'
})

size_brack['Average Math Score'] = size_brack['Average Math Score'].map("{:.2f}".format)
size_brack['Average Reading Score'] = size_brack['Average Reading Score'].map("{:.2f}".format)
size_brack['% Passing Math'] = size_brack['% Passing Math'].map("{:.2f}%".format)
size_brack['% Passing Reading'] = size_brack['% Passing Reading'].map("{:.2f}%".format)
size_brack['Overall Passing Rate'] = size_brack['Overall Passing Rate'].map("{:.2f}%".format)

size_brack
# print(df_schools_new.dtypes)
# df_schools_math = df_schools_new.groupby('School Size').mean()
# df_schools_math

,School Size,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Small (<1000),83.82,83.93,93.55%,96.10%,94.82%
1,Medium (1000-2000),83.37,83.87,93.60%,96.79%,95.20%
2,Large (2000-5000),77.74,81.34,69.96%,82.77%,76.36%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [11]:
type_brack = df_schools_new.groupby('School Type').agg({
    'Average Math Score':'mean',
    'Average Reading Score':'mean',
    'student_name_x':'mean',
    'student_name_y':'mean',
    'Overall Passing Rate':'mean'
}).reset_index()

type_brack = type_brack.rename(columns = {
    'student_name_x':'% Passing Math',
    'student_name_y':'% Passing Reading'
})

type_brack['Average Math Score'] = type_brack['Average Math Score'].map("{:.2f}".format)
type_brack['Average Reading Score'] = type_brack['Average Reading Score'].map("{:.2f}".format)
type_brack['% Passing Math'] = type_brack['% Passing Math'].map("{:.2f}%".format)
type_brack['% Passing Reading'] = type_brack['% Passing Reading'].map("{:.2f}%".format)
type_brack['Overall Passing Rate'] = type_brack['Overall Passing Rate'].map("{:.2f}%".format)

type_brack

,School Type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Charter,83.47,83.90,93.62%,96.59%,95.10%
1,District,76.96,80.97,66.55%,80.80%,73.67%
